In [2]:
import pandas as pd

# Load the original data (df_sheet1) and new data (new_df)
df_sheet1 = pd.read_excel('Fedex working.xlsx', sheet_name='Sheet1')
new_df = pd.read_excel('Export_2024-07-30_134629.xlsx')

# Ensure SKUs are in the correct format
df_sheet1['SKU'] = df_sheet1['SKU'].astype(str)
new_df['Variant SKU'] = new_df['Variant SKU'].astype(str)

# Extract relevant component columns for weight (W (KG)) and length (L (CM))
component_columns = df_sheet1.columns[df_sheet1.columns.str.contains('COMPONENT')]
weight_columns = df_sheet1.columns[df_sheet1.columns.str.contains('W \(KG\)')]
length_columns = df_sheet1.columns[df_sheet1.columns.str.contains('L \(CM\)')]


# Calculate Exceed Weight Quantity and Exceed Length Quantity in df_sheet1
df_sheet1['Exceed Weight Quantity'] = 0
df_sheet1['Exceed Length Quantity'] = 0

for index, row in df_sheet1.iterrows():
    exceed_weight_count = 0
    exceed_length_count = 0
    for w_col, l_col in zip(weight_columns, length_columns):
        if pd.notna(row[w_col]) and row[w_col] > 31:
            exceed_weight_count += 1
        if pd.notna(row[l_col]) and row[l_col] > 122:
            exceed_length_count += 1
    df_sheet1.at[index, 'Exceed Weight Quantity'] = exceed_weight_count
    df_sheet1.at[index, 'Exceed Length Quantity'] = exceed_length_count

# Add 'Weight Type' column
df_sheet1['Weight Type'] = df_sheet1.apply(
    lambda row: 'Volumetric' if row['Vol (KG)'] > row['TOTAL WEIGHT'] else 'True Weight',
    axis=1
)

# Remove duplicates in both datasets based on SKU
df_sheet1 = df_sheet1.drop_duplicates(subset='SKU')
new_df_unique = new_df.drop_duplicates(subset='Variant SKU')

# Merge to get the matching SKUs
merged_df_unique = pd.merge(new_df_unique, df_sheet1[['SKU', 'Exceed Weight Quantity', 'Exceed Length Quantity', 'Weight Type']],
                            left_on='Variant SKU', right_on='SKU', how='inner')

# Merge to include all rows from new_df, adding columns from merged_df_unique where matches exist
updated_df = pd.merge(new_df, merged_df_unique[['Variant SKU', 'Exceed Weight Quantity', 'Exceed Length Quantity', 'Weight Type']],
                      on='Variant SKU', how='left')

# Update Variant Weight where it is 0, using the TOTAL WEIGHT from df_sheet1
updated_df = pd.merge(updated_df, df_sheet1[['SKU', 'TOTAL WEIGHT']], left_on='Variant SKU', right_on='SKU', how='left')

updated_df['Variant Weight'] = updated_df.apply(
    lambda row: row['TOTAL WEIGHT'] if row['Variant Weight'] == 0 and not pd.isna(row['TOTAL WEIGHT']) else row['Variant Weight'],
    axis=1
)

# Display the final rows where Variant Weight was updated from 0
updated_rows = updated_df[(updated_df['Variant Weight'] != 0) & (new_df['Variant Weight'] == 0)]
print(updated_rows[['Variant SKU', 'Variant Weight', 'TOTAL WEIGHT']])


            Variant SKU  Variant Weight  TOTAL WEIGHT
3        TR8PRO-1-TK-EU         40.3209       40.3209
28            RS6-NS-EU          8.3000        8.3000
29            RS6-S3-EU         20.8000       20.8000
33            RS6-S4-EU         16.3000       16.3000
37            RS6-S6-EU         26.9000       26.9000
45     TR160S-WMINV3-EU         78.5000       78.5000
46      TR160S-4INV3-EU         82.3000       82.3000
54     TR160S-DDINV3-EU         77.8000       77.8000
57     TR160S-WMPBNP-EU         64.5000       64.5000
59      TR160S-AINV3-EU         76.3000       76.3000
67     TR160S-WMPBAL-EU         60.5000       60.5000
68   TR160S-L-WMINV3-EU         68.7000       68.7000
69      TR160S-4PBNP-EU         68.3000       68.3000
70    TR160S-L-4INV3-EU         72.5000       72.5000
81      TR160S-4PBAL-EU         64.3000       64.3000
82     TR160S-DDPBNP-EU         63.8000       63.8000
83   TR160S-L-DDINV3-EU         68.0000       68.0000
84      TR160S-APBNP-EU     

In [3]:
updated_df

,Status,Variant ID,Variant SKU,Variant Weight,Variant Weight Unit,Variant Price,Variant Cost,Variant Metafield: custom.height [single_line_text_field],Variant Metafield: custom.width [single_line_text_field],Variant Metafield: custom.length [single_line_text_field],Variant Metafield: custom.weight [single_line_text_field],Variant Metafield: custom.volume [single_line_text_field],Variant Metafield: custom.volumetric_weight [single_line_text_field],round,Shipping price nl,Exceed Weight Quantity,Exceed Length Quantity,Weight Type,SKU,TOTAL WEIGHT
0,Active,42514735399106,TR8PRO-R2R-BUN2-EU,126.6000,kg,4671.0,NaN,0.0,0.0,0.0,0.0,0.0,186.0,0.00064,141.314448,0.0,0.0,Volumetric,TR8PRO-R2R-BUN2-EU,125.6000
1,Active,42514407063746,TR8PRO-R2R-BUN1-EU,126.6000,kg,3901.0,NaN,0.0,0.0,0.0,0.0,0.0,144.0,0.48848,109.480800,0.0,0.0,Volumetric,TR8PRO-R2R-BUN1-EU,92.4000
2,Active,47223492411737,TR80L-1-TK,110.0000,kg,8478.0,NaN,0.0,0.0,0.0,0.0,0.0,117.0,0.00000,89.016312,0.0,1.0,Volumetric,TR80L-1-TK,88.5900
3,Active,47223428612441,TR8PRO-1-TK-EU,40.3209,kg,6241.0,NaN,0.0,0.0,0.0,0.0,0.0,111.0,0.00000,84.468648,0.0,0.0,Volumetric,TR8PRO-1-TK-EU,40.3209
4,Active,47582883807577,SA-09 BUNDLE,8.0000,kg,338.0,NaN,0.0,0.0,0.0,16.5,0.0,84.0,1.26990,64.004160,0.0,0.0,Volumetric,SA-09 BUNDLE,11.7400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119,Draft,49028735402329,nan,0.0000,kg,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,True Weight,nan,0.0000
1120,Draft,49028798546265,nan,0.0000,kg,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,True Weight,nan,0.0000
1121,Draft,49028803494233,nan,0.0000,kg,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,True Weight,nan,0.0000
1122,Draft,49028807328089,nan,0.0000,kg,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,True Weight,nan,0.0000


In [4]:
# Drop the 'SKU' and 'TOTAL WEIGHT' columns from the updated dataframe
final_df = updated_df.drop(columns=['SKU', 'TOTAL WEIGHT'])

# Save the final dataframe to an Excel file for review
final_file_path = 'Updated_SKU_Data.xlsx'
final_df.to_excel(final_file_path, index=False)

final_file_path

'Updated_SKU_Data.xlsx'